In [1]:
import xarray as xr
import numpy as np
import cartopy.util
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
plt.rcParams['mathtext.default'] = 'regular'

In [2]:
# Read data
path_to_geos = Path('../../external_data/GEOS-Chem')
path_to_ukca = Path('../data')
geos = xr.open_dataset(path_to_geos / 'ocean_rono2_2013.nc')
ukca = xr.open_dataset(path_to_ukca / 'xnvtc' / 'xnvtc_o3.nc')
# ukca_srf = xr.open_dataset('/local/mwe14avu/UEA/PhD/models/ukca/data/ukca_coords_n96e_marcus_180705.nc')
# Drop unused UKCA coordinates
ukca = ukca.drop(['forecast_period', 'forecast_reference_time', 'model_level_number', 'level_height', 'sigma', 
                  'forecast_period_bnds', 'time_bnds', 'level_height_bnds', 'sigma_bnds', 'latitude_longitude'])

In [ ]:
# 'GEOS-Chem: ', geos.data_vars, geos.coords
# 'UKCA: ', ukca.data_vars, ukca.coords

In [ ]:
# GEOS-Chem: lon 2.5 lat 2 (at poles step is 1.5): (144, 91) 
# UKCA lon 3.75 lat 2.5 (96, 73)
# GEOS-Chem data doesn't have a land mask and doesn't have fill_values where land is, so need to need to ensure that 
# non-zero fluxes don't appear on land in UKCA and that there aren't any missing grid boxes.
# For that need to compare UKCA surface_altitude variable with GEOS-Chem fluxes.

In [ ]:
# fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0)))
# ax.contourf(geos.MeNO3_emission.lon.values, geos.MeNO3_emission.lat.values, geos.MeNO3_emission[11,...], transform=ccrs.PlateCarree())
# ax.coastlines()
# ax.set_global()

In [ ]:
# geos.MeNO3_emission.lon.values
# ukca.longitude.values

In [ ]:
geos_meono2_emis = geos.MeNO3_emission

In [ ]:
regrdd_meono2_mlc_per_cm2_per_s = geos_meono2_emis.interp(lon=ukca.longitude-180, lat=ukca.latitude,
                                                          method='linear', kwargs=dict(fill_value=0.))
# without fill_value=0 interp() adds nan at all lons at the poles

In [ ]:
# fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0)))
# ax.contourf(intrp.lon.values, intrp.lat.values, intrp[2,...], transform=ccrs.PlateCarree())
# ax.contourf(ukca_srf.surface_altitude)
# ax.coastlines()
# ax.set_global()
# current_cmap = mpl.cm.get_cmap()
# current_cmap.set_bad(color='red')

In [ ]:
geos.MeNO3_emission.units

In [ ]:
# Convert molecules cm-2 s-1 to kg m-2 s-1
# molecules to moles → divide molecules by Na 6.022*1e23 molecules mol-1
# molar mass [g mol-1] 
# molecules  mol                mol g           g             1e-3 kg              kg
#-------------------------, so –----------- = ------- = ------------ = 1e1 ------, thus
# cm2 s        molecules       cm2 s mol    cm2 s    1e-4 m2 s            m2 s
# E/Na*M(RONO2)*1e1

In [ ]:
Na = 6.022*1e23 # [molecules mol-1]
molar_mass_meono2 = 77.0394*1e-3 # [kg mol-1]
molar_mass_etono2 = 91.0660*1e-3 # [kg mol-1]
molar_mass_n = 14.0067*1e-3 # [kg mol-1]
seconds_in_year = 365*24*60*60 # [s]
earth_area = 510.1*1e12 # [m2]

In [ ]:
# Fisher et al (2018) MeONO2 ocean emission 157 Gg N a-1

In [ ]:
import iris
import calendar

In [ ]:
n_days_per_month = np.asarray([calendar.monthrange(2013, i)[1] for i in range(1, 13)])

In [ ]:
gcube = geos_meono2_emis.to_iris()
gcube.coord('Longitude').rename('longitude')
gcube.coord('Latitude').rename('latitude')
gcube.coord(axis='t').rename('time')
lonlat = ['longitude', 'latitude']
for coord in lonlat:
    if not gcube.coord(coord).has_bounds():
        gcube.coord(coord).guess_bounds()
gcube.convert_units('molec/m2/s')

In [ ]:
month_weights = iris.util.broadcast_to_shape(n_days_per_month, gcube.shape, (0,))

In [ ]:
gcube_per_year = (gcube 
                  * iris.coords.AuxCoord(86400, units='s',
                                         long_name='seconds_per_day')).collapsed(['time'],
                                                                                 iris.analysis.SUM,
                                                                                 weights=month_weights)

In [ ]:
norm_weights = iris.analysis.cartography.area_weights(gcube_per_year, normalize=True)
weights = iris.analysis.cartography.area_weights(gcube_per_year, normalize=False)

In [ ]:
gcube_area_sum = gcube_per_year.collapsed(lonlat,
                                          iris.analysis.SUM,
                                          weights=norm_weights)

In [ ]:
gcube_area_sum

In [ ]:
earth_area = iris.cube.Cube(weights.sum(), units='m2')

In [ ]:
na_cube = iris.cube.Cube(Na, units='molec/mol', long_name='avogadro_number')
molar_mass_n_cube = iris.cube.Cube(14, units='g/mol', long_name='molar_mass_of_atomic_nitrogen')

In [ ]:
gcube_global = gcube_area_sum * earth_area * molar_mass_n_cube / na_cube

In [ ]:
gcube_global.convert_units('Gg')

In [ ]:
gcube_global.data

In [ ]:
# regrdd_meono2_kg_per_m2_per_s = regrdd_meono2_mlc_per_cm2_per_s/Na*molar_mass_meono2*1e1

In [ ]:
# regrdd_meono2_kgN_per_m2_per_s = regrdd_meono2_mlc_per_cm2_per_s/Na*molar_mass_n*1e1